In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv

In [2]:
id = []
title = []
writer = []
viewers = []
size = []
time = []
level = []
ingredients = []
description = []

main = []
minor = []
source = []

complete = []

total_quantity = []
main_quantity = []
minor_quantity = []
source_quantity = []
for i in range(25):
    globals()['step{}'.format(i)] = []
    globals()['img{}'.format(i)] = []

In [11]:
def crawling(start,end):
    for page in range(start,end):
        print(page)
        url = 'https://www.10000recipe.com/recipe/list.html?order=reco&page={}'.format(page)
        response = requests.get(url)
        response.encoding = 'UTF-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        ids = soup.find_all('a','thumbnail')

        for id_tmp in ids:
            # 레시피id 찾아서
            id_tmp = id_tmp.get('href').split('e/')[1]
            # 그 id에 따른 상세레시피 페이지 긁어오기
            url_detail = 'https://www.10000recipe.com/recipe/view.html?seq={}'.format(id_tmp)
            response_recipe = requests.get(url_detail)
            response_recipe.encoding = 'UTF-8'
            soup_recipe = BeautifulSoup(response_recipe.text, 'html.parser')


            ## writer(작성자)
            try:
                writer.append(soup_recipe.find('span',{'class':'user_info2_name'}).text.replace(' ',''))
            except:
                writer.append('')
            ## title(제목)
            try:
                title.append(soup_recipe.find('div',{'class':'view2_summary'}).find('h3').text.replace(',',' '))
            except:
                title.append('')
            ## viewers (조회수)
            try:
                viewers.append(int(soup_recipe.find('span',{'class':'hit font_num'}).text.replace(',','')))
            except:
                viewers.append('')
            ## size (몇인분)
            try:
                size.append(int(soup_recipe.find('span',{'class':'view2_summary_info1'}).text.replace('인분','').replace(' 이상','')))
            except:
                size.append('')
            ## time (조리시간)
            try:
                time_temp = int(soup_recipe.find('span',{'class':'view2_summary_info2'}).text.replace('분 이내','').replace('시간 이상','0000'))
                if(time_temp>1000):
                    time_temp = (time_temp/1000)*60
                time.append(time_temp)
            except:
                time.append('')
            ## level (난이도)
            try:
                level.append(soup_recipe.find('span',{'class':'view2_summary_info3'}).text)
            except:
                level.append('')
            ###### description (설명)
            try:
                description.append(soup_recipe.find(id='recipeIntro').text.replace('\n\n\r\n                ','').replace('\n',' '))
            except:
                description.append('')
            ## complete (완성이미지)
            try:
                complete.append(soup_recipe.find(id='main_thumbs').get('src'))
            except:
                complete.append('')


            ## 재료
            total_temp = []
            total_quantity_temp = []
            main_temp = []
            main_quantity_temp =[]
            minor_temp = []
            minor_quantity_temp =[]
            source_temp = []
            source_quantity_temp =[]

            ## '재료' 단어를 포함하면 main으로 포함하지 않으면 minor로 분류. 둘이 합친건 total
            try:
                soup_recipe.find_all('div',{'class':'cont_ingre2'})[0].find_all('ul')
            except:
                try:
                    for i in soup_recipe.find_all('div',{'class':'cont_ingre'})[0].find_all('dl'):
                        total_temp.append(i.find_all('dd')[0].text)
                        total_quantity_temp.append('')
                        if '재료' in i.find('dt').text and '양념' not in i.find('dt').text:
                            main_temp.append(i.find_all('dd')[0].text)
                            main_quantity_temp.append('')
                        elif '양념' in i.find('dt').text or '소스' in i.find('dt').text:
                            source_temp.append(i.find_all('dd')[0].text)
                            source_quantity_temp.append('')
                        else:
                            minor_temp.append(i.find_all('dd')[0].text)
                            minor_quantity_temp.append('')
                except:
                    total_temp.append('')
                    total_quantity_temp.append('')
                    main_temp.append('')
                    main_quantity_temp.append('')
                    source_temp.append('')
                    source_quantity_temp.append('')

            else:
                for i in soup_recipe.find_all('div',{'class':'cont_ingre2'})[0].find_all('ul'):
                    for j in i.find_all('li'):
                        total_temp.append(j.text.split('   ')[0])
                        total_quantity_temp.append(j.find('span',{'class':'ingre_unit'}).text)
                    if '재료' in i.find('b').text:
                        for j in i.find_all('li'):
                            main_temp.append(j.text.split('   ')[0])
                            main_quantity_temp.append(j.find('span',{'class':'ingre_unit'}).text)
                    elif '양념' in i.find('b').text or '소스' in i.find('b').text:
                        for j in i.find_all('li'):
                            source_temp.append(j.text.split('   ')[0])
                            source_quantity_temp.append(j.find('span',{'class':'ingre_unit'}).text)
                    else:
                        for j in i.find_all('li'):
                            minor_temp.append(j.text.split('   ')[0])
                            minor_quantity_temp.append(j.find('span',{'class':'ingre_unit'}).text)


            ingredients.append(total_temp)
            main.append(main_temp)
            minor.append(minor_temp)
            source.append(source_temp)

            total_quantity.append(total_quantity_temp)
            main_quantity.append(main_quantity_temp)
            minor_quantity.append(minor_quantity_temp)
            source_quantity.append(source_quantity_temp)

            ## 레시피 step0~step24, img0~img24 채우기  , 없을경우 null

            try:

                step_size = len(soup_recipe.find_all('div',{'class':'view_step_cont'}))

                for i in range(25):
                    if(i<step_size):
                        try:
                            globals()['img{}'.format(i)].append(soup_recipe.find_all('div',{'class':'view_step_cont'})[i].find_all('div')[1].find('img').get('src'))
                            globals()['step{}'.format(i)].append(soup_recipe.find_all('div',{'class':'view_step_cont'})[i].find('div').text)

                        except:
                            globals()['img{}'.format(i)].append('')
                            globals()['step{}'.format(i)].append(soup_recipe.find_all('div',{'class':'view_step_cont'})[i].find('div').text)
                    else:
                        globals()['step{}'.format(i)].append('')
                        globals()['img{}'.format(i)].append('')
            except:
                for i in range(25):
                    globals()['step{}'.format(i)].append('')
                    globals()['img{}'.format(i)].append('')
            id.append(id_tmp)
    
    recipe = {'id':id, 'title':title, 'writer':writer, 'viewers':viewers, 'size':size, 'time':time, 'level':level,
         'ingredients':ingredients, 'description':description, 'total_quantity': total_quantity, 'main':main, 'main_quantity':main_quantity, 'minor':minor, 'complete':complete,
         'minor_quantity':minor_quantity, 'source':source, 'source_quantity':source_quantity, 'step0':step0, 'img0':img0, 'step1':step1, 'img1':img1,'step2':step2, 'img2':img2,'step3':step3, 'img3':img3,'step4':step4, 'img4':img4,'step5':step5, 'img5':img5,'step6':step6, 'img6':img6,'step7':step7, 'img7':img7,'step8':step8, 'img8':img8,'step9':step9, 'img9':img9, 'step10':step10, 'img10':img10, 'step11':step11, 'img11':img11,'step12':step12, 'img12':img12,'step13':step13, 'img13':img13,'step14':step14, 'img14':img14,'step15':step15, 'img15':img15,'step16':step16, 'img16':img16,'step17':step17, 'img17':img17,'step18':step18, 'img18':img18,'step19':step19, 'img19':img19,  'step20':step20, 'img20':img20, 'step21':step21, 'img21':img21,'step22':step22, 'img22':img22,'step23':step23, 'img23':img23,'step24':step24, 'img24':img24}
    recipe_df = pd.DataFrame(recipe)
    recipe_df.to_csv("crawling7.csv", mode='w')

In [12]:
crawling(2260,2401)

2260
2261
2262
2263
2264
2265
2266
2267
2268
2269
2270
2271
2272
2273
2274
2275
2276
2277
2278
2279
2280
2281
2282
2283
2284
2285
2286
2287
2288
2289
2290
2291
2292
2293
2294
2295
2296
2297
2298
2299
2300
2301
2302
2303
2304
2305
2306
2307
2308
2309
2310
2311
2312
2313
2314
2315
2316
2317
2318
2319
2320
2321
2322
2323
2324
2325
2326
2327
2328
2329
2330
2331
2332
2333
2334
2335
2336
2337
2338
2339
2340
2341
2342
2343
2344
2345
2346
2347
2348
2349
2350
2351
2352
2353
2354
2355
2356
2357
2358
2359
2360
2361
2362
2363
2364
2365
2366
2367
2368
2369
2370
2371
2372
2373
2374
2375
2376
2377
2378
2379
2380
2381
2382
2383
2384
2385
2386
2387
2388
2389
2390
2391
2392
2393
2394
2395
2396
2397
2398
2399
2400


In [24]:
dataset = pd.read_csv("crawling2.csv")
dataset

,Unnamed: 0,id,title,writer,viewers,size,time,level,ingredients,description,...,step20,img20,step21,img21,step22,img22,step23,img23,step24,img24
0,0,6832078,집밥 백선생 '백종원 콩나물 볶음밥',SophiePark,6048,2.0,30.0,초급,"['콩나물', '계란', '밥', '송송 썬 대파', '다진 당근', '양념', '...",콩나물로 만들 수 있는 간단한 볶음밥이예요,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,6909351,보글보글 간단한찌개 끓이기 ! 순두부찌개 ;),만개의레시피,65080,2.0,20.0,아무나,"['순두부', '양파', '대파', '물', '간장', '굴소스', '고춧가루', ...",보들보들 순두부가 잔뜩 들어간 칼칼한 순두부찌개 끓이기 : ) 세상의 모든 레시피 ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,6845728,계란피자만들기 - 아이간식,먹깨비레시피,5440,3.0,30.0,초급,"['계란', '피자치즈', '햄', '양파', '옥수수콘', '파프리카', '양송이...",계란은 완전식품이라 불릴정도로 몸에 좋은거 아 시죠?~~ 밀가루 도우없이 비쥬얼 완...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,6919073,[무화과 레시피] 꿀이 흐르는 프렌치 토스트,찬스키친,740,2.0,20.0,초급,"['식빵', '계란', '우유', '꿀', '계피가루', '무화과', '생크림', ...",세겹의 식빵으로 만든 프렌치 토스트 입니다! 빵 사이사이 꿀이 들어가 더욱더 달콤하...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,6920939,술안주로 좋은 차돌박이찜,마튈다,3918,2.0,10.0,초급,"['차돌박이', '부추', '숙주', '당근', '칼국수', '대파', '다진마늘'...",숙주를 넣어 아삭한 차돌박이찜 소개해 볼게요 : ),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,6849901,백종원의 오므라이스소스 - 집밥백선생2 10회,집밥백선생,55461,6.0,30.0,아무나,"['식용유', '간돼지고기', '다진 양파', '다진 당근', '다진 대파', '다...",· 집밥백선생2 10회 · 방송일 : 2016.05.24 · 동영상 출처 : 집밥백...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,6884045,눈과 입이 즐거운 맛있는 한끼/삼색 파프리카 계란밥구이,쿡따라,2895,3.0,30.0,초급,"['삼색 파프리카', '흑미밥', '계란', '깻잎', '모짜렐라치즈', '맛술',...","알록달록 삼색파프리카속에 밥과 계란, 치즈가 들어가 있어 영양만점 파프리카 컵밥입니...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,6879290,밤고구마로 고구마 스콘 만들기,효썸,5246,6.0,90.0,초급,"['박력분', '설탕', '소금', '베이킹파우더', '차가운 버터', '차가운 우...","고구마를 깍뚝 썰어서 사용하면 고구마 맛을 잘 느낄 수 있지만, 지저분해 보일 수도...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,6833929,"""명란젓 스파게티""♪",서재인,13782,1.0,15.0,아무나,"['스파게티', '명란젓', '통마늘', '양파', '청양고추', '불고추', '식...",젓갈종류 중 비싸지만 그 맛에 한번 빠지면 헤어나오지 못한다는 명란젓!! 그냥 먹어...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,6900285,실패 없이 순대볶음 만드는 법,동주,18941,2.0,30.0,아무나,"['순대', '깻잎', '청양고추', '양파', '당근', '양배추', '들깨가루'...",오랜만에 어머니와 쉬는 날이 겹쳐서 어머니를 위해 순대볶음을 만들었습니다. 순대볶음...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def str_to_list(str):
    str = str.replace("'", "").replace("[", "").replace("]", "").replace(', ',",")
    words = str.split(',')
    word_list=[]
    for word in words:
        word_list.append(word)
    
    return word_list

In [35]:
a = dataset.loc[0][10].replace('"','').replace('[','').replace(']','').replace("'",'').split(', ')
a[0]

'2줌'